In [1]:
import pandas as pd
import numpy as np
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g7
g7=rdflib.Graph()
g7.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_SmartCity_7.nt',format='nt')
print(len(g7))

102571


In [3]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [4]:
#************Change Path to files************
state_dist_code_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/StateDistrictCodes.csv",engine='python') 
state_dist_code_data=state_dist_code_data.replace(np.nan,"",regex=True)

In [5]:
state_dist_code_data.head(5)

,Name,2001-StateCode,2001-DistrictCode,2011-StateCode,2011-DistrictCode,Unnamed: 5
0,JAMMU AND KASHMIR,1,0,1,0,
1,Anantnag,1,6,1,14,
2,Budgam,1,2,1,2,
3,Bandipore,1,2,1,9,
4,Baramulla,1,2,1,8,


In [6]:
district_code_data=state_dist_code_data.loc[state_dist_code_data['2011-DistrictCode']!=0,:]
district_code_data.reset_index(inplace=True)
district_code_data=district_code_data.drop(['index'],axis=1)

In [7]:
state_code_data=state_dist_code_data.loc[state_dist_code_data['2011-DistrictCode']==0,:]
state_code_data.reset_index(inplace=True)
state_code_data=state_code_data.drop(['index'],axis=1)

In [8]:
#!pip install geocoder

In [9]:
import geocoder
def get_spatial(entity,typ):
    if typ.lower()=="district":
        g = geocoder.geonames(entity,key='codradmin',country = 'IN',featureClass = 'A', featureCode = 'ADM2')
    else:
        g = geocoder.geonames(entity,key='codradmin',country = 'IN',featureClass = 'A', featureCode = 'ADM1')
    return(g.geonames_id)

In [10]:
#Creating smaller graph for State&UTs
g9=rdflib.Graph()
for s in g7.subjects(skos.prefLabel,Literal("State & Union Territory")):
        district_uri=s
        break

for s,p,o in g7.triples((None,bhorg.hasName,None)):
    if (s, org.classification,district_uri) in g7:
        g9.add((s,p,o))
print(len(g9))

36


In [11]:
for s,p,o in g9.triples((None,None,None)):
    code_2011=state_code_data.loc[state_code_data['Name']==str(o).upper(),'2011-StateCode'].values
    code_2001=state_code_data.loc[state_code_data['Name']==str(o).upper(),'2001-StateCode'].values
    if len(code_2001)>0:
        c_11=code_2011[0]
        c_1=code_2001[0]
        state_code_data.loc[state_code_data['Name']==str(o).upper(),'log']='found'
        state_code_2001=Literal(c_1,datatype=URIRef('http://semanticwebindia.in/bharathi/ont/org/StateCensusCode2001'))
        state_code_2011=Literal(c_11,datatype=URIRef('http://semanticwebindia.in/bharathi/ont/org/StateCensusCode2011'))
        g7.add((s,bhorg.hasCode,state_code_2011)) #change to g7
        g7.add((s,bhorg.hasCode,state_code_2001)) #change to g7
        spatial_id=get_spatial(str(o),"State & Union Territory")
        g7.add((s,bhorg.hasSpatial,Literal(spatial_id))) #change to g7
    else:
        print(s,p,o)

In [12]:
print(len(g9))

36


In [13]:
#Creating district graph for smaller iterations for District
g8=rdflib.Graph()
for s in g7.subjects(skos.prefLabel,Literal("District")):
        district_uri=s
        break

for s,p,o in g7.triples((None,bhorg.hasName,None)):
    if (s, org.classification,district_uri) in g7:
        g8.add((s,p,o))
print(len(g8))

705


In [14]:
#District Code
#g9=rdflib.Graph()
for s,p,o in g8.triples((None,None,None)):
    code_2011=district_code_data.loc[district_code_data['Name']==str(o),'2011-DistrictCode'].values
    code_2001=district_code_data.loc[district_code_data['Name']==str(o),'2001-DistrictCode'].values
    if len(code_2001)>0:
        c_11=code_2011[0]
        c_1=code_2001[0]
        district_code_data.loc[district_code_data['Name']==str(o),'log']='found'
        district_code_2001=Literal(c_1,datatype=URIRef('http://semanticwebindia.in/bharathi/ont/org/DistrictCensusCode2001'))
        district_code_2011=Literal(c_11,datatype=URIRef('http://semanticwebindia.in/bharathi/ont/org/DistrictCensusCode2011'))
        g7.add((s,bhorg.hasCode,district_code_2001))#change to g7
        g7.add((s,bhorg.hasCode,district_code_2011))#change to g7
        spatial_id=get_spatial(str(o),"district")
        g7.add((s,bhorg.hasSpatial,Literal(spatial_id)))#change to g7

In [15]:
print(len(g8))

705


In [16]:
#Testing district Code
i=0
u=URIRef('http://semanticwebindia.in/bharathi/res/org/14705')
for s,p,o in g8.triples((u,None,None)):
    print(s,p,o)
    i+=1
    if i>20:
        break

In [17]:
district_code_data.head(10)

,Name,2001-StateCode,2001-DistrictCode,2011-StateCode,2011-DistrictCode,Unnamed: 5,log
0,Anantnag,1,6,1,14,,found
1,Budgam,1,2,1,2,,found
2,Bandipore,1,2,1,9,,found
3,Baramulla,1,2,1,8,,found
4,Doda,1,9,1,16,,found
5,Ganderbal,1,2,1,11,,found
6,Jammu,1,13,1,21,,found
7,Kargil,1,8,1,4,,found
8,Kathua,1,14,1,7,,found
9,Kishtwar,1,9,1,18,,found


In [18]:
#Concatenating State and district Code data
df=[state_code_data,district_code_data]
total_data=pd.concat(df)

In [19]:
#total_data.to_csv('C:/Users/SESA456791/Desktop/Ontology-Bh/State_District_data_w_log.csv',encoding='utf8',index=False)
state_code_data.to_csv('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/state_data_w_log.csv',encoding='utf8',index=False)

In [20]:
print(len(g7))

104476


In [21]:
g7.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_State_District_Code_8.nt')

In [22]:
for s,p,o in g7.triples((None,bhorg.hasName,Literal("Telangana"))):
    for s1,p1,o1 in g7.triples((s,skos.Notation,None)):
        print(s1,p1,o1)

 ****************Not to run District Code****************
district_code_data['log']=""
for s in g7.subjects(skos.prefLabel,Literal("District")):
        district_uri=s
for i in range(district_code_data.shape[0]):
    district_nam=district_code_data.iloc[i,0].title()
    for s in g7.subjects(bhorg.hasName,Literal(district_nam)):
        if (s, org.classification,district_uri) in g7:
            #print('found',s,val,i)
            district_code_data.iloc[i,5]="found"
            district_code_2001=Literal(district_nam,datatype=URIRef('http://www.semanticwebindia.in/bharathi/ont/org/StateCensusCode2001'))
            district_code_2011=Literal(district_nam,datatype=URIRef('http://www.semanticwebindia.in/bharathi/ont/org/StateCensusCode2011'))
            g7.add((s,skos.Notation,district_code_2001))
            g7.add((s,skos.Notation,district_code_2011))
            spatial_id=get_spatial(state_nam,"district")
            g7.add((s,bhorg.hasSpatial,Literal(spatial_id)))
        else:
            district_code_data.iloc[i,5]="Could not find a corresponding org in org vocab"

#************No tto be run State Code************
state_code_data['log']=""
for s in g7.subjects(skos.prefLabel,Literal("State & UTs")):
        state_ut_uri=s
        break
for i in range(state_code_data.shape[0]):
    state_nam=state_code_data.iloc[i,0].title()
    for s in g7.subjects(bhorg.hasName,Literal(state_nam)):
        if (s, org.classification,state_ut_uri) in g7:
            #print('found',s,val,i)
            state_code_data.iloc[i,5]="found"
            state_code_2001=Literal(state_nam,datatype=URIRef('http://www.semanticwebindia.in/bharathi/ont/org/StateCensusCode2001'))
            state_code_2011=Literal(state_nam,datatype=URIRef('http://www.semanticwebindia.in/bharathi/ont/org/StateCensusCode2011'))
            g7.add((s,skos.Notation,state_code_2001))
            g7.add((s,skos.Notation,state_code_2011))
            spatial_id=get_spatial(state_nam,"State & UTs")
            g7.add((s,bhorg.hasSpatial,Literal(spatial_id)))
        else:
            state_code_data.iloc[i,5]="Could not find a corresponding org in org vocab"

In [23]:
#for Log
for s,p,o in g8.triples((None,None,None)):
    if (s,skos.Notation,None) not in g7:
        for s1,p1,o1 in g8.triples((s,bhorg.hasName,None)):
            print(s1,p1,o1)

http://semanticwebindia.in/bharathi/res/org/14584 http://semanticwebindia.in/bharathi/ont/org/hasName Bongaigaon
http://semanticwebindia.in/bharathi/res/org/17035 http://semanticwebindia.in/bharathi/ont/org/hasName Balaghat
http://semanticwebindia.in/bharathi/res/org/18304 http://semanticwebindia.in/bharathi/ont/org/hasName Amritsar
http://semanticwebindia.in/bharathi/res/org/110443 http://semanticwebindia.in/bharathi/ont/org/hasName Lohardaga
http://semanticwebindia.in/bharathi/res/org/14900 http://semanticwebindia.in/bharathi/ont/org/hasName Chandigarh
http://semanticwebindia.in/bharathi/res/org/15051 http://semanticwebindia.in/bharathi/ont/org/hasName Raipur
http://semanticwebindia.in/bharathi/res/org/16008 http://semanticwebindia.in/bharathi/ont/org/hasName Fatehabad
http://semanticwebindia.in/bharathi/res/org/110864 http://semanticwebindia.in/bharathi/ont/org/hasName Uttara Kannada (Karwar)
http://semanticwebindia.in/bharathi/res/org/18305 http://semanticwebindia.in/bharathi/ont/o

http://semanticwebindia.in/bharathi/res/org/110848 http://semanticwebindia.in/bharathi/ont/org/hasName Davangere
http://semanticwebindia.in/bharathi/res/org/19457 http://semanticwebindia.in/bharathi/ont/org/hasName Udham Singh Nagar
http://semanticwebindia.in/bharathi/res/org/18537 http://semanticwebindia.in/bharathi/ont/org/hasName Alwar
http://semanticwebindia.in/bharathi/res/org/16010 http://semanticwebindia.in/bharathi/ont/org/hasName Hisar
http://semanticwebindia.in/bharathi/res/org/110053 http://semanticwebindia.in/bharathi/ont/org/hasName Hooghly
http://semanticwebindia.in/bharathi/res/org/15771 http://semanticwebindia.in/bharathi/ont/org/hasName Porbandar
http://semanticwebindia.in/bharathi/res/org/18552 http://semanticwebindia.in/bharathi/ont/org/hasName Jaisalmer
http://semanticwebindia.in/bharathi/res/org/18041 http://semanticwebindia.in/bharathi/ont/org/hasName Kandhamal
http://semanticwebindia.in/bharathi/res/org/19790 http://semanticwebindia.in/bharathi/ont/org/hasName Ra